In [1]:
import pandas as pd
import numpy as np

In [2]:
train_dataset = np.load('./data/train_image_pop.npy')
test_dataset = np.load('./data/test_image_pop.npy')

print(train_dataset.shape)
print(test_dataset.shape)

(4320, 10, 20)
(1440, 10, 20)


# Sampler

In [3]:
# def sampler(data, lag=8, step=1, temp=False):
#     """This function makes samples of the time series data
#     args:
#     - data : (# of data, height, width)
#     - lag : the length of sampling
#     - step : (step)-ahead forecasting label
#     return: 
#     - data_x (# of samples, height, width, lag)
#     - data_y (# of samples, height, width, 1)
#     """
#     num_row = len(data)
#     data_x, data_y = [], []
#     for idx in range(num_row-lag-step):
#         y = np.array(data[idx+lag : idx+lag+step])
#         if not temp:
#             x = np.transpose(data[idx : idx+lag], [1,2,0])
#             data_x.append(x)
#             y = np.transpose(y, [1,2,0])
        
#         data_y.append(y)

#     print("Sampler Return", np.shape(data_x), np.shape(data_y))
                        
#     if not temp:
#         return np.array(data_x), np.array(data_y)
#     else:
#         return np.array(data_y)

In [4]:
def sampler_stamp(data, stamp, lag, step, temp=False):

    num_row = len(data)
    data_x, data_y = [], []
    for idx in range(num_row-stamp*lag-step):
        y = np.array(data[stamp*lag+idx : stamp*lag+idx+step])
        if not temp:
            x = np.transpose(data[[stamp*i+idx for i in range(lag)],:], [1,2,0])
            data_x.append(x)
            y = np.transpose(y, [1,2,0])
        data_y.append(y)
    print("Sampler Return", np.shape(data_x), np.shape(data_y))
    
    if not temp:
        return np.array(data_x), np.array(data_y)
    else:
        return np.array(data_y)

# Make Temporal Information

In [5]:
print("Train shape: ", np.shape(train_dataset), ", Test shape: ", np.shape(test_dataset))

# Setting Some Parameters 
num_train, num_test = np.shape(train_dataset)[0], np.shape(test_dataset)[0]
num_row = num_train + num_test
print('num_row: ', num_row)

Train shape:  (4320, 10, 20) , Test shape:  (1440, 10, 20)
num_row:  5760


In [6]:
### Initialize numpy array of temporal information (one-hot encoding)
datasets_min_30 = np.zeros([num_row, 48])
datasets_dow = np.zeros([num_row, 7])

# 더미화 하기
# 30 mins, and day-of-week index are calculated below
for i in range(num_row):
    idx_30 = int(int(i)%48)
    idx_dow = int(int(i/48)%7)
    datasets_min_30[i,idx_30] = 1
    datasets_dow[i, idx_dow] = 1

In [7]:
def train_test_split(data, idx):
    return data[:idx], data[idx:]

In [8]:
# Split Train & Test Period

train_index = num_train #144 # 120 days and 144 time index
min_30_train, min_30_test = train_test_split(datasets_min_30, train_index)
dow_train, dow_test = train_test_split(datasets_dow, train_index)

print(min_30_train.shape, min_30_test.shape)
print(dow_train.shape, dow_test.shape)

(4320, 48) (1440, 48)
(4320, 7) (1440, 7)


# Save Final Data

In [9]:
def save_data_stamp(STAMP, LAG, STEP):
    # train 
    x_train, y_train = sampler_stamp(train_dataset, stamp=STAMP, lag=LAG, step=1, temp=False)    
    min_30_train_y = sampler_stamp(min_30_train, stamp=STAMP, lag=LAG, step=1, temp=True)
    dow_train_y = sampler_stamp(dow_train, stamp=STAMP, lag=LAG, step=1, temp=True)
    temporal_train = np.concatenate((dow_train_y, min_30_train_y), axis=-1)
    
    np.savez(f'./data/x_train_stamp{STAMP}_lag{LAG}.npz', x_train)
    np.savez(f'./data/y_train_stamp{STAMP}_lag{LAG}.npz', y_train)
    np.savez(f'./data/temporal_train_stamp{STAMP}_lag{LAG}.npz', temporal_train)
    
    # test
    x_test, y_test = sampler_stamp(test_dataset, stamp=STAMP, lag=LAG, step=STEP, temp=False)
    min_30_test_y = sampler_stamp(min_30_test, stamp=STAMP, lag=LAG, step=STEP, temp=True)
    dow_test_y = sampler_stamp(dow_test, stamp=STAMP, lag=LAG, step=STEP, temp=True)
    temporal_test = np.concatenate((dow_test_y, min_30_test_y), axis=-1)

    np.savez(f'./data/x_test_stamp{STAMP}_lag{LAG}_step{STEP}.npz', x_test)
    np.savez(f'./data/y_test_stamp{STAMP}_lag{LAG}_step{STEP}.npz', y_test)
    np.savez(f'./data/temporal_test_stamp{STAMP}_lag{LAG}_step{STEP}.npz', temporal_test)

In [10]:
lag_list = [2*(i+1) for i in range(12)]
for lag in lag_list:
    save_data_stamp(STAMP=1, LAG=lag, STEP=1)

Sampler Return (4317, 10, 20, 2) (4317, 10, 20, 1)
Sampler Return (0,) (4317, 1, 48)
Sampler Return (0,) (4317, 1, 7)
Sampler Return (1437, 10, 20, 2) (1437, 10, 20, 1)
Sampler Return (0,) (1437, 1, 48)
Sampler Return (0,) (1437, 1, 7)
Sampler Return (4315, 10, 20, 4) (4315, 10, 20, 1)
Sampler Return (0,) (4315, 1, 48)
Sampler Return (0,) (4315, 1, 7)
Sampler Return (1435, 10, 20, 4) (1435, 10, 20, 1)
Sampler Return (0,) (1435, 1, 48)
Sampler Return (0,) (1435, 1, 7)
Sampler Return (4313, 10, 20, 6) (4313, 10, 20, 1)
Sampler Return (0,) (4313, 1, 48)
Sampler Return (0,) (4313, 1, 7)
Sampler Return (1433, 10, 20, 6) (1433, 10, 20, 1)
Sampler Return (0,) (1433, 1, 48)
Sampler Return (0,) (1433, 1, 7)
Sampler Return (4311, 10, 20, 8) (4311, 10, 20, 1)
Sampler Return (0,) (4311, 1, 48)
Sampler Return (0,) (4311, 1, 7)
Sampler Return (1431, 10, 20, 8) (1431, 10, 20, 1)
Sampler Return (0,) (1431, 1, 48)
Sampler Return (0,) (1431, 1, 7)
Sampler Return (4309, 10, 20, 10) (4309, 10, 20, 1)
Samp

In [12]:
time_unit = [0.5,1,2,3,4,6,8,12,24]
stamp_list = [int(2*i) for i in time_unit]
step_list = [int(48/i) for i in stamp_list]
lag_list = [2*(i+1) for i in range(12)]

for i in range(len(time_unit)): # time unit에 따라
    for j in range(len(lag_list)): # lag에 따라
        save_data_stamp(STAMP=stamp_list[i], LAG=lag_list[j], STEP=step_list[i])

Sampler Return (4317, 10, 20, 2) (4317, 10, 20, 1)
Sampler Return (0,) (4317, 1, 48)
Sampler Return (0,) (4317, 1, 7)
Sampler Return (1390, 10, 20, 2) (1390, 10, 20, 48)
Sampler Return (0,) (1390, 48, 48)
Sampler Return (0,) (1390, 48, 7)
Sampler Return (4315, 10, 20, 4) (4315, 10, 20, 1)
Sampler Return (0,) (4315, 1, 48)
Sampler Return (0,) (4315, 1, 7)
Sampler Return (1388, 10, 20, 4) (1388, 10, 20, 48)
Sampler Return (0,) (1388, 48, 48)
Sampler Return (0,) (1388, 48, 7)
Sampler Return (4313, 10, 20, 6) (4313, 10, 20, 1)
Sampler Return (0,) (4313, 1, 48)
Sampler Return (0,) (4313, 1, 7)
Sampler Return (1386, 10, 20, 6) (1386, 10, 20, 48)
Sampler Return (0,) (1386, 48, 48)
Sampler Return (0,) (1386, 48, 7)
Sampler Return (4311, 10, 20, 8) (4311, 10, 20, 1)
Sampler Return (0,) (4311, 1, 48)
Sampler Return (0,) (4311, 1, 7)
Sampler Return (1384, 10, 20, 8) (1384, 10, 20, 48)
Sampler Return (0,) (1384, 48, 48)
Sampler Return (0,) (1384, 48, 7)
Sampler Return (4309, 10, 20, 10) (4309, 10,

Sampler Return (1340, 10, 20, 22) (1340, 10, 20, 12)
Sampler Return (0,) (1340, 12, 48)
Sampler Return (0,) (1340, 12, 7)
Sampler Return (4223, 10, 20, 24) (4223, 10, 20, 1)
Sampler Return (0,) (4223, 1, 48)
Sampler Return (0,) (4223, 1, 7)
Sampler Return (1332, 10, 20, 24) (1332, 10, 20, 12)
Sampler Return (0,) (1332, 12, 48)
Sampler Return (0,) (1332, 12, 7)
Sampler Return (4307, 10, 20, 2) (4307, 10, 20, 1)
Sampler Return (0,) (4307, 1, 48)
Sampler Return (0,) (4307, 1, 7)
Sampler Return (1420, 10, 20, 2) (1420, 10, 20, 8)
Sampler Return (0,) (1420, 8, 48)
Sampler Return (0,) (1420, 8, 7)
Sampler Return (4295, 10, 20, 4) (4295, 10, 20, 1)
Sampler Return (0,) (4295, 1, 48)
Sampler Return (0,) (4295, 1, 7)
Sampler Return (1408, 10, 20, 4) (1408, 10, 20, 8)
Sampler Return (0,) (1408, 8, 48)
Sampler Return (0,) (1408, 8, 7)
Sampler Return (4283, 10, 20, 6) (4283, 10, 20, 1)
Sampler Return (0,) (4283, 1, 48)
Sampler Return (0,) (4283, 1, 7)
Sampler Return (1396, 10, 20, 6) (1396, 10, 20,

Sampler Return (4079, 10, 20, 20) (4079, 10, 20, 1)
Sampler Return (0,) (4079, 1, 48)
Sampler Return (0,) (4079, 1, 7)
Sampler Return (1196, 10, 20, 20) (1196, 10, 20, 4)
Sampler Return (0,) (1196, 4, 48)
Sampler Return (0,) (1196, 4, 7)
Sampler Return (4055, 10, 20, 22) (4055, 10, 20, 1)
Sampler Return (0,) (4055, 1, 48)
Sampler Return (0,) (4055, 1, 7)
Sampler Return (1172, 10, 20, 22) (1172, 10, 20, 4)
Sampler Return (0,) (1172, 4, 48)
Sampler Return (0,) (1172, 4, 7)
Sampler Return (4031, 10, 20, 24) (4031, 10, 20, 1)
Sampler Return (0,) (4031, 1, 48)
Sampler Return (0,) (4031, 1, 7)
Sampler Return (1148, 10, 20, 24) (1148, 10, 20, 4)
Sampler Return (0,) (1148, 4, 48)
Sampler Return (0,) (1148, 4, 7)
Sampler Return (4287, 10, 20, 2) (4287, 10, 20, 1)
Sampler Return (0,) (4287, 1, 48)
Sampler Return (0,) (4287, 1, 7)
Sampler Return (1405, 10, 20, 2) (1405, 10, 20, 3)
Sampler Return (0,) (1405, 3, 48)
Sampler Return (0,) (1405, 3, 7)
Sampler Return (4255, 10, 20, 4) (4255, 10, 20, 1)

Sampler Return (3455, 10, 20, 18) (3455, 10, 20, 1)
Sampler Return (0,) (3455, 1, 48)
Sampler Return (0,) (3455, 1, 7)
Sampler Return (575, 10, 20, 18) (575, 10, 20, 1)
Sampler Return (0,) (575, 1, 48)
Sampler Return (0,) (575, 1, 7)
Sampler Return (3359, 10, 20, 20) (3359, 10, 20, 1)
Sampler Return (0,) (3359, 1, 48)
Sampler Return (0,) (3359, 1, 7)
Sampler Return (479, 10, 20, 20) (479, 10, 20, 1)
Sampler Return (0,) (479, 1, 48)
Sampler Return (0,) (479, 1, 7)
Sampler Return (3263, 10, 20, 22) (3263, 10, 20, 1)
Sampler Return (0,) (3263, 1, 48)
Sampler Return (0,) (3263, 1, 7)
Sampler Return (383, 10, 20, 22) (383, 10, 20, 1)
Sampler Return (0,) (383, 1, 48)
Sampler Return (0,) (383, 1, 7)
Sampler Return (3167, 10, 20, 24) (3167, 10, 20, 1)
Sampler Return (0,) (3167, 1, 48)
Sampler Return (0,) (3167, 1, 7)
Sampler Return (287, 10, 20, 24) (287, 10, 20, 1)
Sampler Return (0,) (287, 1, 48)
Sampler Return (0,) (287, 1, 7)
